# Chat Writer

本 Notebook 快速概述了如何开始使用 Writer 的 [Chat](/docs/concepts/chat_models/)。

Writer 提供多种聊天模型。您可以在 [Writer 文档](https://dev.writer.com/home)中找到有关其最新模型、成本、上下文窗口和支持的输入类型的详细信息。


## 概述

### 集成详情
| Class                                                                                                                    | Package          | Local | Serializable | JS support |                                        Package downloads                                         |                                        Package latest                                         |
|:-------------------------------------------------------------------------------------------------------------------------|:-----------------| :---: | :---: |:----------:|:------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------:|
| [ChatWriter](https://github.com/writer/langchain-writer/blob/main/langchain_writer/chat_models.py#L308) | [langchain-writer](https://pypi.org/project/langchain-writer/) |      ❌       |                                       ❌                                       | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-writer?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-writer?style=flat-square&label=%20) |
### 模型功能
| [工具调用](/docs/how_to/tool_calling) | 结构化输出 | JSON 模式 | 图片输入 | 音频输入 | 视频输入 | [Token 级流式输出](/docs/how_to/chat_streaming/) | 原生异步 |         [Token 使用情况](/docs/how_to/chat_token_usage_tracking/)          | Logprobs |
| :---: |:-----------------:| :---: | :---: |  :---: | :---: | :---: | :---: |:--------------------------------:|:--------:|
| ✅ |         ❌         | ❌ | ❌ | ❌ | ❌ | ✅ | ✅ |                ✅                 |    ❌     |

### 凭证

注册 [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) 并按照此 [快速入门](https://dev.writer.com/api-guides/quickstart) 获取 API 密钥。然后，设置 WRITER_API_KEY 环境变量：

In [ ]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

如果你想自动跟踪你的模型调用，你也可以通过取消下面这行注释来设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### 安装

`ChatWriter` 可从 `langchain-writer` 包获取。请使用以下命令进行安装：

In [ ]:
%pip install -qU langchain-writer

### 实例化

现在我们可以实例化模型对象来生成聊天补全：

In [ ]:
from langchain_writer import ChatWriter

llm = ChatWriter(
    model="palmyra-x-004",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

## 用法

要使用该模型，你需要传入一个消息列表并调用 `invoke` 方法：

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

然后，你可以访问消息的内容：

In [ ]:
print(ai_msg.content)

## 流式输出

你也可以流式输出响应。首先，创建一个流：

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming. Sing a song about it"),
]
ai_stream = llm.stream(messages)
ai_stream

然后，遍历流以获取块：

In [ ]:
for chunk in ai_stream:
    print(chunk.content, end="")

## 工具调用

Writer 模型，例如 Palmyra X 004，支持 [工具调用](https://dev.writer.com/api-guides/tool-calling)，它允许您描述工具及其参数。模型将返回一个 JSON 对象，其中包含要调用的工具以及该工具的输入。

### 绑定工具

使用 `ChatWriter.bind_tools`，您可以轻松地将 Pydantic 类、字典模式、LangChain 工具，甚至函数传递给模型作为工具。在底层，它们被转换为工具模式，其外观如下：
```
{
    "name": "...",
    "description": "...",
    "parameters": {...}  # JSONSchema
}
```
这些会在每次模型调用时传递。

例如，要使用一个获取给定位置的天气的工具，您可以定义一个 Pydantic 类并将其传递给 `ChatWriter.bind_tools`：

In [ ]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


llm.bind_tools([GetWeather])

然后，您可以使用该工具调用模型：

In [ ]:
ai_msg = llm.invoke(
    "what is the weather like in New York City",
)
ai_msg

最终，您可以访问工具调用，并着手执行您的函数：

In [ ]:
print(ai_msg.tool_calls)

### 关于工具绑定的注意事项

`ChatWriter.bind_tools()` 方法不会创建带有绑定工具的新实例，而是将接收到的 `tools` 和 `tool_choice` 存储在初始类实例属性中，以便在使用 `ChatWriter` 调用时将它们作为参数传递给 Palmyra LLM。这种方法支持不同的工具类型，例如 `function` 和 `graph`。`Graph` 是其中一个远程调用的 Writer Palmyra 工具。更多信息请访问我们的[文档](https://dev.writer.com/api-guides/knowledge-graph#knowledge-graph)。

有关 LangChain 中工具 ব্যবহারের更多信息，请访问[LangChain 工具调用文档](https://python.langchain.com/docs/concepts/tool_calling/)。

## 批量处理

你也可以批量处理请求并设置 `max_concurrency`：

In [ ]:
ai_batch = llm.batch(
    [
        "How to cook pancakes?",
        "How to compose poem?",
        "How to run faster?",
    ],
    config={"max_concurrency": 3},
)
ai_batch

然后，迭代批处理以获取结果：

In [ ]:
for batch in ai_batch:
    print(batch.content)
    print("-" * 100)

## 异步使用

以上所有功能（调用、流式处理、批量处理、工具调用）都支持异步使用。

## Prompt 模板

[Prompt 模板](https://python.langchain.com/docs/concepts/prompt_templates/) 帮助将用户输入和参数转换成语言模型的指令。你可以像这样在 `ChatWriter` 中使用 prompt 模板：

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

## API 参考
有关 ChatWriter 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/writer/chat_models/langchain_writer.chat_models.ChatWriter.html#langchain_writer.chat_models.ChatWriter)。

## 其他资源
您可以在 [Writer 文档](https://dev.writer.com/home) 中找到有关 Writer 模型（包括成本、上下文窗口和支持的输入类型）以及工具的信息。